<a href="https://colab.research.google.com/github/mchlltene/Machine_Learning-_Algorithms/blob/main/k-Nearest%20Neighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# Import Python Packages**



In [2]:
##########################################################
# 1. IMPORT ALL PACKAGES
##########################################################
import pandas as pd
from sklearn.metrics import confusion_matrix
import math
from collections import Counter
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
import numpy as np
import string
from sklearn import svm
from sklearn.tree import export_graphviz
import re
import nltk
nltk.download("stopwords"); # Filtering (Stopword Removal)
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# import pydot
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import multilabel_confusion_matrix
from statistics import mean 

import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
#PACKAGE UNTUK CONNECT G-DRIVE N G-COLLAB 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#PANDAS LIBRARY UNTUK DATAFRAME
import pandas as pd
dataset = ('/content/drive/MyDrive/DataScrap/test_data.csv')
df_review = pd.read_csv(dataset, sep=';')
df_review.head()


,userName,score,label,date,content
0,Lia Muliawati,1,Negatif,16/07/2022,Sayang sekali pesanan ga bisa di batalkan saat...
1,Popi Efrianti,1,Negatif,16/07/2022,"Barang yg diinginkan banyak yg tdk tersedia, p..."
2,AdI Prayetno,1,Negatif,16/07/2022,Lemottt xxx..... Padahal jaringan baguss
3,Ayu Wulandary,1,Negatif,16/07/2022,"IKLAN NYA AJA GERCEP,TAPI KADANG TIDAK DI ANTA..."
4,Aditya Putra Pratama,1,Negatif,16/07/2022,"Mau download aja susah banget, pdhl jaringan b..."


In [5]:
##########################################################
# 2. LOAD TESTING AND TRAINING DATASET
dateset_train =("/content/drive/MyDrive/64373/DATA/DATA_TESTING.csv")
dateset_test = ("/content/drive/MyDrive/64373/DATA/DATA_TESTINGG.csv")

df_train = pd.read_csv(dateset_train , sep=';')
df_test = pd.read_csv(dateset_test, sep=';')

df_train = df_train.dropna(subset=['content'])
df_test = df_test.dropna(subset=['content'])

# Check all classes/labels in training data
all_label = set(df_train['score'].tolist())
dst_train = Counter(df_train['score'])
df_train

,score,label,at,content
0,1,Negatif,27/05/2022,"Aplikasi perlu diperbaiki lagi,loading banget ..."
1,1,Negatif,27/05/2022,kenapa kok sekarang susah dibuka yah.. Udh seb...
2,1,Negatif,27/05/2022,kok produk rokok hilang??
3,1,Negatif,27/05/2022,sebener nya bagus sih.. tp skrg jadi minus.. k...
4,1,Negatif,26/05/2022,Tukar poin gak bisa2. Udah instal berulang kx....
...,...,...,...,...
995,0,Positif,20/06/2022,"Aplikasi alfagift ini sangat membantu sekali, ..."
996,0,Positif,20/06/2022,Aplikasi alfamart yg mudah untuk melakukan bel...
997,0,Positif,20/06/2022,"Aplikasi bagus, banyak promo ÃƒÂ°Ã…Â¸Ã¢â‚¬ËœÃ‚..."
998,0,Positif,20/06/2022,Aplikasi belanja andalan ketika malas keluar r...


In [6]:
# Function to remove punctuations
def remove_punctuations(text):
  for sym_punc in string.punctuation:
    text = text.replace(sym_punc, ' ') 
    text = re.sub(' +', ' ', text) # replace multiple spaces with single space
  return text;

# Function to remove duplicate characters
def remove_duplicate_chars(text):
  pattern = r'(.)\1+' # (.) any character repeated (\+) more than
  repl = r'\1'        # replace it once
  text = re.sub(pattern, repl, text);
  return text;

def normalization(text):
  return text.replace("maf", "maaf").replace("ko", "kenapa").replace("kok", "kenapa").replace("g", "tidak").replace("pdhl", "padahal").replace("in", "ini").replace("bnget","banget").replace("blnja","belanja").replace("ga","tidak").replace("tp","tapi").replace("pdhal","padahal").replace("utk","untuk").replace("sy","saya").replace("tdk","tidak").replace("apk","aplikasi").replace("ngak","tidak").replace("dlu","dulu").replace("gak","tidak").replace("klu","kalau").replace("tlg","tolong").replace("gak","tidak").replace("adain","adakan").replace("jd","jadi").replace("skrng","sekarang").replace("bgus","bagus").replace("nerima","terima").replace("kenpa","kenapa").replace("uda","sudah").replace("skrg","sekarang").replace("mnggu","menunggu").replace("krja","kerja").replace("plg","pulang").replace("happy","senang").replace("good","bagus").replace("msuk","masuk").replace("pd","pada").replace("aja","saja").replace("bgaus","bagus").replace("klau","kalau").replace("jd","jadi").replace("sdh","sudah").replace("sdg","sedang").replace("donload","download").replace("udah","sudah").replace("lamaaaaa","lama").replace("klo","kalau").replace("lncar","lancar").replace("ngirim","kirim").replace("banyk","banyak").replace("smg","semoga").replace("blanja","belanja").replace("ngga","tidak").replace("lgsg","langsung").replace("gada","tidak ada").replace("pgn","ingin").replace("gada","tidak ada").replace("ktawa","tertawa").replace("aq","aku").replace("no","nomor").replace("ok","oke").replace("oky","oke").replace("oklah","oke").replace("lemottt","lemot").replace("liat","lihat").replace("bs","bisa").replace("eror","error").replace("ngk","tidak").replace("yg","yang").replace("apk","aplikasi").replace("soalx","soalnya").replace("sbg","sebagai").replace("gw","saya").replace("pda","pada").replace("vouche","vocher").replace("mlh","malah").replace("slah","salah").replace("slah","salah").replace("susa","susah").replace("yh","yah").replace("usefull","membantu").replace("belnjar","belanja").replace("sippp","sip").replace("akuh","saya").replace("alfagiftttt","alfagift").replace("tq","terima kasih").replace("kt","kata").replace("bguuuus","bagus").replace("helpfull","membantu")

# Data preprocessingk
df_train['content'] = df_train['content'].str.replace('\d+', '', regex=True) # Remove numbers from string
df_train['content'] = df_train['content'].apply(lambda x:x.lower()) # Change Strings to lowercase
df_train["content"] = df_train['content'].apply(remove_punctuations) # Remove punctuations (Annotation Removal)
df_train["content"] = df_train['content'].apply(remove_duplicate_chars) # Remove repeated characters
df_train['content'] = df_train['content'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii')) # Remove emojis
# df_train['content'] = df_train['content'].apply(normalization) 

#TEST DATA
# Data preprocessing
df_test['content'] = df_test['content'].str.replace('\d+', '', regex=True) # Remove numbers from string
df_test['content'] = df_test['content'].apply(lambda x:x.lower()) # Change Strings to lowercase
df_test["content"] = df_test['content'].apply(remove_punctuations) # Remove punctuations (Annotation Removal)
df_test["content"] = df_test['content'].apply(remove_duplicate_chars) # Remove repeated characters
df_test['content'] = df_test['content'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii')) # Remove emojis
# df_test['content'] = df_test['content'].apply(normalization) 

df_train.head(10)


,score,label,at,content
0,1,Negatif,27/05/2022,aplikasi perlu diperbaiki lagi loading banget ...
1,1,Negatif,27/05/2022,kenapa kok sekarang susah dibuka yah udh sebul...
2,1,Negatif,27/05/2022,kok produk rokok hilang
3,1,Negatif,27/05/2022,sebener nya bagus sih tp skrg jadi minus karen...
4,1,Negatif,26/05/2022,tukar poin gak bisa udah instal berulang kx ma...
5,1,Negatif,26/05/2022,coba di aplikasi ini ada fitur chat dengan tok...
6,1,Negatif,26/05/2022,harga di tempat tokonya sama di aplikasi apa m...
7,1,Negatif,26/05/2022,masa vocher di struk cashback pas daftar via a...
8,1,Negatif,26/05/2022,sinyal udah bagus kecepatan juga bagus muter m...
9,1,Negatif,26/05/2022,test pesan antar dr jam sore ditguin smpe jam ...


In [7]:
from warnings import filterwarnings
# Function to load filter to list (berisi kata-kata yang akan di filter)
def load_indonesia_filter():
  # get stop words
  filtering = stopwords.words("indonesian");
  # append additional stopword
  filtering.extend(["yg","untuk", "dg", "rt", "dgn", "ny", "d", 'klo', 'deh','sekarang','ku','jam','rumah','kan','nya','member','alfa','gift',"aplikasi",
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 'tokonya', 'toko', 'bnget','ini',"dulu","kalau","jadi","sekarang","kerja","pulang","pada","sedang","download","langsung","ingin","tertawa",
                       "nomor","lihat","aplikasi","sebagai","saya","voucher","malah","yah","alfagift",'untuk', 'ga', 'krn', 'nya', 'nih', 'sih', 'emg', 'kk',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'yaaa', 'ya','nomor', 'aktif','titik','jd', 'jgn', 'sdh', 'aja', 'ka', 't', 'user','versi','nyg', 'hehe', 'yaa', 'lihat', 'nan', 'loh', 'rs','telepon','x',
                       '&amp', 'yah', 'covid','dirawat','aplikasi','apk','aplikasinya','sehari', 'kasir',"ponta","saat","atau", "anak", "pencet","login", "di", "hp", "masa", "nomor","trus","nya","masa","struk",
                       "qt", "bayar","kan","keranjang","checkout","dr","sistem","alfagift","aplikasi","alfa","kaya","baru","apk","down","hbs","jam","saat","di","app",
                      "nya","ke","sih","alfa","youtube","nafsu","belanja","kali","barang",'ambil','pas','dateng','kurir','outlet','vmg','bekasi','utara','dah','scan','tapi','butuh','besok','sat','memesan', 
                      'pake','belanjan','keranjang','edit','pokok','mending','memory','recipt','dihapus','sya','blanja','dpt','points',"point", 'kebijakanya','pelangan','dimana','foucher','cashback','klaim','sedah',
                      'alfamart','faktur','dint','nt', 'daftar','nyampe','tu','ada','beli','es','krim','carikan','leleh','kupon','min','order','memenuhi','syarat','eh','produk','produknya','kaya','pengiriman','list','brp','alamat'])
  filtering.remove("padahal") 
  filtering.remove('ada')
  filtering.remove('agak') 
  filtering.remove('tapi') 
  filtering.remove('juga') 
  filtering.remove('seharusnya') 
  filtering.remove('bisa') 
  filtering.remove('kenapa') 
  filtering.remove('tambah') 
  filtering.remove('sangat') 
  return filtering;

# load filter words
filter_words = load_indonesia_filter();

# remove stop words (filtering)
df_train['content'] = df_train['content'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in filter_words))
df_train = df_train.dropna(subset=['content']) # Remove Missing Values
df_train.head()

# remove stop words (filtering)
df_test['content'] = df_test['content'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in filter_words))
df_test = df_test.dropna(subset=['content']) # Remove Missing Values
df_test.head(10)

,score,label,at,content,Unnamed: 4
0,1,Negatif,27/05/2022,diperbaiki loading banget yng dicari ngk deliv...,NaN
1,1,Negatif,27/05/2022,kenapa susah dibuka udh sebulan gk bisa pke al...,NaN
2,1,Negatif,27/05/2022,rokok hilang,NaN
3,1,Negatif,27/05/2022,sebener bagus tp skrg minus bsa rokok delivery,NaN
4,1,Negatif,26/05/2022,tukar poin gak bisa udah instal berulang kx,NaN
5,1,Negatif,26/05/2022,coba fitur chat kadang pesen diantar kadang ba...,NaN
6,1,Negatif,26/05/2022,harga beda bh swety xl bh sgm gr vanila bh por...,NaN
7,1,Negatif,26/05/2022,vocher via gak nominal voucherny tertera fikti...,NaN
8,1,Negatif,26/05/2022,sinyal udah bagus kecepatan juga bagus muter m...,NaN
9,1,Negatif,26/05/2022,test pesan sore ditguin smpe mlm blm juga pesa...,NaN


In [8]:

##########################################################
# 3. SAVE CORPUS DATA AS A FILE
##########################################################

# load corpus data / kumpulan teks / dictionary (corpus data is used to generate a dictionary)
corpus_data = df_train.append(df_test) # from testing and training data
corpus = corpus_data['content'].to_numpy()

# 1-gram
model_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
model_vectorizer.fit(corpus)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", model_vectorizer.vocabulary_)

# saving python object as pkl (vocabulary)
joblib.dump(model_vectorizer, "/content/drive/MyDrive/64373/Models/vectorizer_Esnsemble_class.pkl")



Vocabulary:  {'diperbaiki': 534, 'loading': 1236, 'banget': 150, 'yng': 2403, 'dicari': 469, 'ngk': 1536, 'delivery': 420, 'udah': 2338, 'cantumkan': 336, 'kagak': 950, 'bisa': 256, 'simpan': 2017, 'kenapa': 1042, 'susah': 2125, 'dibuka': 465, 'udh': 2339, 'sebulan': 1956, 'gk': 768, 'pke': 1786, 'alfafift': 41, 'halaman': 812, 'putih': 1849, 'mulu': 1473, 'padahal': 1626, 'ful': 712, 'kuota': 1172, 'tolong': 2292, 'bulanan': 316, 'rokok': 1896, 'hilang': 840, 'sebener': 1954, 'bagus': 133, 'tp': 2297, 'skrg': 2037, 'minus': 1433, 'bsa': 304, 'tukar': 2323, 'poin': 1801, 'gak': 722, 'instal': 876, 'berulang': 227, 'kx': 1176, 'coba': 369, 'fitur': 699, 'chat': 358, 'kadang': 948, 'pesen': 1766, 'diantar': 440, 'kejelasan': 1012, 'pesananya': 1764, 'engak': 674, 'harga': 819, 'beda': 175, 'bh': 246, 'swety': 2128, 'xl': 2397, 'sgm': 1998, 'gr': 785, 'vanila': 2360, 'pororo': 1812, 'milk': 1429, 'botol': 289, 'ml': 1441, 'ditoko': 583, 'total': 2295, 'an': 69, 'kasih': 975, 'pembelian': 

['/content/drive/MyDrive/64373/Models/vectorizer_Esnsemble_class.pkl']

In [9]:
##########################################################
# 4. FEATURE EXTRACTION TECHNIQUES
##########################################################
# load object vectorizer (vocabulary)
vectorizer = joblib.load("/content/drive/MyDrive/64373/Models/vectorizer_Esnsemble_class.pkl")

# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)


Vocabulary:  {'diperbaiki': 534, 'loading': 1236, 'banget': 150, 'yng': 2403, 'dicari': 469, 'ngk': 1536, 'delivery': 420, 'udah': 2338, 'cantumkan': 336, 'kagak': 950, 'bisa': 256, 'simpan': 2017, 'kenapa': 1042, 'susah': 2125, 'dibuka': 465, 'udh': 2339, 'sebulan': 1956, 'gk': 768, 'pke': 1786, 'alfafift': 41, 'halaman': 812, 'putih': 1849, 'mulu': 1473, 'padahal': 1626, 'ful': 712, 'kuota': 1172, 'tolong': 2292, 'bulanan': 316, 'rokok': 1896, 'hilang': 840, 'sebener': 1954, 'bagus': 133, 'tp': 2297, 'skrg': 2037, 'minus': 1433, 'bsa': 304, 'tukar': 2323, 'poin': 1801, 'gak': 722, 'instal': 876, 'berulang': 227, 'kx': 1176, 'coba': 369, 'fitur': 699, 'chat': 358, 'kadang': 948, 'pesen': 1766, 'diantar': 440, 'kejelasan': 1012, 'pesananya': 1764, 'engak': 674, 'harga': 819, 'beda': 175, 'bh': 246, 'swety': 2128, 'xl': 2397, 'sgm': 1998, 'gr': 785, 'vanila': 2360, 'pororo': 1812, 'milk': 1429, 'botol': 289, 'ml': 1441, 'ditoko': 583, 'total': 2295, 'an': 69, 'kasih': 975, 'pembelian': 

In [10]:
# Generate feature [test and train] from dictionary
# Testing data
test_data = vectorizer.transform(df_test['content']).toarray()
feature_test = pd.DataFrame(data = test_data, columns = vectorizer.get_feature_names_out())

# training data
train_data = vectorizer.transform(df_train['content']).toarray()
feature_train = pd.DataFrame(data = train_data, columns = vectorizer.get_feature_names_out())

feature_test

,abang,abangnya,abis,abisan,ac,aceh,acuhin,ad,adain,admin,...,yaini,yak,yauda,yea,yes,yng,you,yuk,zom,zu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.544607,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [11]:
##########################################################
# 4. ADD CLASS TO TESTING AND TRAINING DATA
##########################################################
# ['POSITIF','NEGATIF', 'NETRAL'], [0, 1, 2]

# train
class_train = df_train['score'];
feature_train['score'] = class_train

# test
class_test = df_test['score'];
feature_test['score'] = class_test

feature_test = feature_test.dropna() # Drop rows if missing values
feature_train = feature_train.dropna() # Drop rows if missing values

feature_test

,abang,abangnya,abis,abisan,ac,aceh,acuhin,ad,adain,admin,...,yak,yauda,yea,yes,yng,you,yuk,zom,zu,score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.544607,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0


In [12]:
##########################################################
# 6. DEFINE X AND Y VARIABLES
##########################################################
# # Testing data
x_test = feature_test.iloc[:, :-1]
y_test = feature_test['score'].tolist()

# Training data
x_train = feature_train.iloc[:, :-1]
y_train = feature_train['score'].tolist()

# Print shape of the Datasets
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
# print(feature_test.shape[0], 'test samples')

x_train shape: (1000, 2408)
1000 train samples


In [13]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [14]:
# MODEL MENGGUNAKAN KNN
model1 = KNeighborsClassifier()
model1.fit(x_train, y_train)
pred = model1.predict(x_test)

In [15]:
##########################################################
# 7. APPLY THE TRAINED LEARNER TO TEST NEW DATA
##########################################################
# # Apply the trained perceptron to make prediction of test data
y_pred = model1.predict(x_test)


##########################################################
# 8. CONFUSION MATRIX
##########################################################
# Actual and predicted classes
lst_actual_class = y_test
lst_predicted_class = pred

# label ['POSITIF', 'NETRAL','NEGATIF'], [0, 1, 2]
lst_classes = [0, 1, 2]

# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];

# Loop for each taget label
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Predicted Performance of Digit Label/Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    precision = round(tp/(tp + fp), 3)
    fscore = round((2 * ((precision * sensitivity) / (precision + sensitivity))), 3)
    mcc = round((((tp * tn)-(fp * fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))),3)

    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(fscore);
    store_mcc.append(mcc);
    
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(fscore))
    print("MCC: {0}\n".format(mcc))

Predicted Performance of Digit Label/Class: 0
TP=119, FP=31, TN=669, FN=181
Sensitivity: 0.397
Specificity: 0.956
Accuracy: 0.788
Balanced Accuracy: 0.676
Precision: 0.793
F1-Score: 0.529
MCC: 0.452

Predicted Performance of Digit Label/Class: 1
TP=31, FP=6, TN=694, FN=269
Sensitivity: 0.103
Specificity: 0.991
Accuracy: 0.725
Balanced Accuracy: 0.547
Precision: 0.838
F1-Score: 0.183
MCC: 0.23

Predicted Performance of Digit Label/Class: 2
TP=393, FP=420, TN=180, FN=7
Sensitivity: 0.982
Specificity: 0.3
Accuracy: 0.573
Balanced Accuracy: 0.641
Precision: 0.483
F1-Score: 0.648
MCC: 0.355



In [16]:
##########################################################
# 8. OVERALL - FINAL PREDICTION PERFORMANCE
##########################################################
print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))

Overall Performance Prediction:
Sensitivity: 49.4%
Specificity: 74.9%
Accuracy: 69.5333%
Balanced Accuracy: 62.1333%
Precision: 70.4667%
F1-Score: 45.3333%
MCC: 0.3457



In [17]:
# calculate execution time
import time
start_time = time.time()
print("Time : %s seconds." % (time.time() - start_time))

Time : 9.226799011230469e-05 seconds.


In [18]:
pred_proba = model1.predict_proba(x_test);
pred_proba

array([[0.4, 0.2, 0.4],
       [0. , 0.2, 0.8],
       [0. , 0.2, 0.8],
       ...,
       [0.4, 0.2, 0.4],
       [0.6, 0.2, 0.2],
       [0.6, 0.2, 0.2]])